# 🧪 Survey Synthetic Dataset Generator — Week 3 Task

In [ ]:

import os, re, json, time, uuid, math, random
from datetime import datetime, timedelta
from typing import List, Dict, Any
import numpy as np, pandas as pd
import pandera.pandas as pa
random.seed(7); np.random.seed(7)
print("✅ Base libraries ready. Pandera available:", pa is not None)


In [ ]:

def extract_strict_json(text: str):
    """Improved JSON extraction with multiple fallback strategies"""
    if text is None:
        raise ValueError("Empty model output.")
    
    t = text.strip()
    
    # Strategy 1: Direct JSON parsing
    try:
        obj = json.loads(t)
        if isinstance(obj, list):
            return obj
        elif isinstance(obj, dict):
            for key in ("rows","data","items","records","results"):
                if key in obj and isinstance(obj[key], list):
                    return obj[key]
            if all(isinstance(k, str) and k.isdigit() for k in obj.keys()):
                return [obj[k] for k in sorted(obj.keys(), key=int)]
    except json.JSONDecodeError:
        pass
    
    # Strategy 2: Extract JSON from code blocks
    if t.startswith("```"):
        t = re.sub(r"^```(?:json)?\s*|\s*```$", "", t, flags=re.IGNORECASE|re.MULTILINE).strip()
    
    # Strategy 3: Find JSON array in text
    start, end = t.find('['), t.rfind(']')
    if start == -1 or end == -1 or end <= start:
        raise ValueError("No JSON array found in model output.")
    
    t = t[start:end+1]
    
    # Strategy 4: Fix common JSON issues
    t = re.sub(r",\s*([\]}])", r"\1", t)  # Remove trailing commas
    t = re.sub(r"\bNaN\b|\bInfinity\b|\b-Infinity\b", "null", t)  # Replace NaN/Infinity
    t = t.replace("\u00a0", " ").replace("\u200b", "")  # Remove invisible characters
    
    try:
        return json.loads(t)
    except json.JSONDecodeError as e:
        raise ValueError(f"Could not parse JSON: {str(e)}. Text: {t[:200]}...")


## 1) Configuration

In [ ]:

CFG = {
    "rows": 800,
    "datetime_range": {"start": "2024-01-01", "end": "2025-10-01", "fmt": "%Y-%m-%d %H:%M:%S"},
    "fields": [
        {"name": "response_id", "type": "uuid4"},
        {"name": "respondent_id", "type": "int", "min": 10000, "max": 99999},
        {"name": "submitted_at", "type": "datetime"},
        {"name": "country", "type": "enum", "values": ["KE","UG","TZ","RW","NG","ZA"], "probs": [0.50,0.10,0.12,0.05,0.15,0.08]},
        {"name": "language", "type": "enum", "values": ["en","sw"], "probs": [0.85,0.15]},
        {"name": "device", "type": "enum", "values": ["android","ios","web"], "probs": [0.60,0.25,0.15]},
        {"name": "age", "type": "int", "min": 18, "max": 70},
        {"name": "gender", "type": "enum", "values": ["female","male","nonbinary","prefer_not_to_say"], "probs": [0.49,0.49,0.01,0.01]},
        {"name": "education", "type": "enum", "values": ["primary","secondary","diploma","bachelor","postgraduate"], "probs": [0.08,0.32,0.18,0.30,0.12]},
        {"name": "income_band", "type": "enum", "values": ["low","lower_mid","upper_mid","high"], "probs": [0.28,0.42,0.23,0.07]},
        {"name": "completion_seconds", "type": "float", "min": 60, "max": 1800, "distribution": "lognormal"},
        {"name": "attention_passed", "type": "bool"},
        {"name": "q_quality", "type": "int", "min": 1, "max": 5},
        {"name": "q_value", "type": "int", "min": 1, "max": 5},
        {"name": "q_ease", "type": "int", "min": 1, "max": 5},
        {"name": "q_support", "type": "int", "min": 1, "max": 5},
        {"name": "nps", "type": "int", "min": 0, "max": 10},
        {"name": "is_detractor", "type": "bool"}
    ]
}
print("Loaded config for", CFG["rows"], "rows and", len(CFG["fields"]), "fields.")


## 2) Helpers

In [ ]:

def sample_enum(values, probs=None, size=None):
    values = list(values)
    if probs is None:
        probs = [1.0 / len(values)] * len(values)
    return np.random.choice(values, p=probs, size=size)

def sample_numeric(field_cfg, size=1):
    t = field_cfg["type"]
    if t == "int":
        lo, hi = int(field_cfg["min"]), int(field_cfg["max"])
        dist = field_cfg.get("distribution", "uniform")
        if dist == "uniform":
            return np.random.randint(lo, hi + 1, size=size)
        elif dist == "normal":
            mu = (lo + hi) / 2.0
            sigma = (hi - lo) / 6.0
            out = np.random.normal(mu, sigma, size=size)
            return np.clip(out, lo, hi).astype(int)
        else:
            return np.random.randint(lo, hi + 1, size=size)
    elif t == "float":
        lo, hi = float(field_cfg["min"]), float(field_cfg["max"])
        dist = field_cfg.get("distribution", "uniform")
        if dist == "uniform":
            return np.random.uniform(lo, hi, size=size)
        elif dist == "normal":
            mu = (lo + hi) / 2.0
            sigma = (hi - lo) / 6.0
            return np.clip(np.random.normal(mu, sigma, size=size), lo, hi)
        elif dist == "lognormal":
            mu = math.log(max(1e-3, (lo + hi) / 2.0))
            sigma = 0.75
            out = np.random.lognormal(mu, sigma, size=size)
            return np.clip(out, lo, hi)
        else:
            return np.random.uniform(lo, hi, size=size)
    else:
        raise ValueError("Unsupported numeric type")

def sample_datetime(start: str, end: str, size=1, fmt="%Y-%m-%d %H:%M:%S"):
    s = datetime.fromisoformat(start)
    e = datetime.fromisoformat(end)
    total = int((e - s).total_seconds())
    r = np.random.randint(0, total, size=size)
    return [(s + timedelta(seconds=int(x))).strftime(fmt) for x in r]


## 3) Rule-based Generator

In [ ]:

def generate_rule_based(CFG: Dict[str, Any]) -> pd.DataFrame:
    n = CFG["rows"]
    dt_cfg = CFG.get("datetime_range", {"start":"2024-01-01","end":"2025-10-01","fmt":"%Y-%m-%d %H:%M:%S"})
    data = {}
    for f in CFG["fields"]:
        name, t = f["name"], f["type"]
        if t == "uuid4":
            data[name] = [str(uuid.uuid4()) for _ in range(n)]
        elif t in ("int","float"):
            data[name] = sample_numeric(f, size=n)
        elif t == "enum":
            data[name] = sample_enum(f["values"], f.get("probs"), size=n)
        elif t == "datetime":
            data[name] = sample_datetime(dt_cfg["start"], dt_cfg["end"], size=n, fmt=dt_cfg["fmt"])
        elif t == "bool":
            data[name] = np.random.rand(n) < 0.9  # 90% True
        else:
            data[name] = [None]*n
    df = pd.DataFrame(data)

    # Derive NPS roughly from likert questions
    if set(["q_quality","q_value","q_ease","q_support"]).issubset(df.columns):
        likert_avg = df[["q_quality","q_value","q_ease","q_support"]].mean(axis=1)
        df["nps"] = np.clip(np.round((likert_avg - 1.0) * (10.0/4.0) + np.random.normal(0, 1.2, size=n)), 0, 10).astype(int)

    # Heuristic target: is_detractor more likely when completion high & attention failed
    if "is_detractor" in df.columns:
        base = 0.25
        comp = df.get("completion_seconds", pd.Series(np.zeros(n)))
        attn = pd.Series(df.get("attention_passed", np.ones(n))).astype(bool)
        boost = (comp > 900).astype(int) + (~attn).astype(int)
        p = np.clip(base + 0.15*boost, 0.01, 0.95)
        df["is_detractor"] = np.random.rand(n) < p

    return df

df_rule = generate_rule_based(CFG)
df_rule.head()


## 4) Validation (Pandera optional)

In [ ]:

def build_pandera_schema(CFG):
    if pa is None:
        return None
    cols = {}
    for f in CFG["fields"]:
        t, name = f["type"], f["name"]
        if t == "int": cols[name] = pa.Column(int)
        elif t == "float": cols[name] = pa.Column(float)
        elif t == "enum": cols[name] = pa.Column(object)
        elif t == "datetime": cols[name] = pa.Column(object)
        elif t == "uuid4": cols[name] = pa.Column(object)
        elif t == "bool": cols[name] = pa.Column(bool)
        else: cols[name] = pa.Column(object)
    return pa.DataFrameSchema(cols) if pa is not None else None

def validate_df(df, CFG):
    schema = build_pandera_schema(CFG)
    if schema is None:
        return df, {"engine":"basic","valid_rows": len(df), "invalid_rows": 0}
    try:
        v = schema.validate(df, lazy=True)
        return v, {"engine":"pandera","valid_rows": len(v), "invalid_rows": 0}
    except Exception as e:
        print("Validation error:", e)
        return df, {"engine":"pandera","valid_rows": len(df), "invalid_rows": 0, "notes": "Non-strict mode."}

validated_rule, report_rule = validate_df(df_rule, CFG)
print(report_rule)
validated_rule.head()


## 5) Save

In [ ]:

from pathlib import Path
out = Path("data"); out.mkdir(exist_ok=True)
ts = datetime.utcnow().strftime("%Y%m%dT%H%M%SZ")
csv_path = out / f"survey_rule_{ts}.csv"
validated_rule.to_csv(csv_path, index=False)
print("Saved:", csv_path.as_posix())


## 6) Optional: LLM Generator (JSON mode, retry & strict parsing)

In [ ]:
# Fixed LLM Generation Functions
def create_survey_prompt(CFG, n_rows=50):
    """Create a clear, structured prompt for survey data generation"""
    fields_desc = []
    for field in CFG['fields']:
        name = field['name']
        field_type = field['type']
        
        if field_type == 'int':
            min_val = field.get('min', 0)
            max_val = field.get('max', 100)
            fields_desc.append(f"  - {name}: integer between {min_val} and {max_val}")
        elif field_type == 'float':
            min_val = field.get('min', 0.0)
            max_val = field.get('max', 100.0)
            fields_desc.append(f"  - {name}: float between {min_val} and {max_val}")
        elif field_type == 'enum':
            values = field.get('values', [])
            fields_desc.append(f"  - {name}: one of {values}")
        elif field_type == 'bool':
            fields_desc.append(f"  - {name}: boolean (true/false)")
        elif field_type == 'uuid4':
            fields_desc.append(f"  - {name}: UUID string")
        elif field_type == 'datetime':
            fmt = field.get('fmt', '%Y-%m-%d %H:%M:%S')
            fields_desc.append(f"  - {name}: datetime string in format {fmt}")
        else:
            fields_desc.append(f"  - {name}: {field_type}")
    
    prompt = f"""Generate {n_rows} rows of realistic survey response data.

Schema:
{chr(10).join(fields_desc)}

CRITICAL REQUIREMENTS:
- Return a JSON object with a "responses" key containing an array
- Each object in the array must have all required fields
- Use realistic, diverse values for survey responses
- No trailing commas
- No comments or explanations

Output format: JSON object with "responses" array containing exactly {n_rows} objects.

Example structure:
{{
  "responses": [
    {{
      "response_id": "uuid-string",
      "respondent_id": 12345,
      "submitted_at": "2024-01-01 12:00:00",
      "country": "KE",
      "language": "en",
      "device": "android",
      "age": 25,
      "gender": "female",
      "education": "bachelor",
      "income_band": "upper_mid",
      "completion_seconds": 300.5,
      "attention_passed": true,
      "q_quality": 4,
      "q_value": 3,
      "q_ease": 5,
      "q_support": 4,
      "nps": 8,
      "is_detractor": false
    }},
    ...
  ]
}}

IMPORTANT: Return ONLY the JSON object with "responses" key, nothing else."""
    
    return prompt

def repair_truncated_json(content):
    """Attempt to repair truncated JSON responses"""
    content = content.strip()
    
    # If it starts with { but doesn't end with }, try to close it
    if content.startswith('{') and not content.endswith('}'):
        # Find the last complete object in the responses array
        responses_start = content.find('"responses": [')
        if responses_start != -1:
            # Find the last complete object
            brace_count = 0
            last_complete_pos = -1
            in_string = False
            escape_next = False
            
            for i, char in enumerate(content[responses_start:], responses_start):
                if escape_next:
                    escape_next = False
                    continue
                    
                if char == '\\':
                    escape_next = True
                    continue
                    
                if char == '"' and not escape_next:
                    in_string = not in_string
                    continue
                    
                if not in_string:
                    if char == '{':
                        brace_count += 1
                    elif char == '}':
                        brace_count -= 1
                        if brace_count == 0:
                            last_complete_pos = i
                            break
            
            if last_complete_pos != -1:
                # Truncate at the last complete object and close the JSON
                repaired = content[:last_complete_pos + 1] + '\n  ]\n}'
                print(f"🔧 Repaired JSON: truncated at position {last_complete_pos}")
                return repaired
    
    return content

def fixed_llm_generate_batch(CFG, n_rows=50):
    """Fixed LLM generation with better prompt and error handling"""
    if not os.getenv('OPENAI_API_KEY'):
        print("No OpenAI API key, using rule-based fallback")
        tmp = dict(CFG); tmp['rows'] = n_rows
        return generate_rule_based(tmp)
    
    try:
        from openai import OpenAI
        client = OpenAI()
        
        prompt = create_survey_prompt(CFG, n_rows)
        
        print(f"🔄 Generating {n_rows} survey responses with LLM...")
        
        # Calculate appropriate max_tokens based on batch size
        # Roughly 200-300 tokens per row, with some buffer
        estimated_tokens = n_rows * 300 + 500  # Buffer for JSON structure
        max_tokens = min(max(estimated_tokens, 2000), 8000)  # Between 2k-8k tokens
        
        print(f"📊 Using max_tokens: {max_tokens} (estimated: {estimated_tokens})")
        
        response = client.chat.completions.create(
            model='gpt-4o-mini',
            messages=[
                {'role': 'system', 'content': 'You are a data generation expert. Generate realistic survey data in JSON format. Always return complete, valid JSON.'},
                {'role': 'user', 'content': prompt}
            ],
            temperature=0.3,
            max_tokens=max_tokens,
            response_format={'type': 'json_object'}
        )
        
        content = response.choices[0].message.content
        print(f"📝 Raw response length: {len(content)} characters")
        
        # Check if response appears truncated
        if not content.strip().endswith('}') and not content.strip().endswith(']'):
            print("⚠️ Response appears truncated, attempting repair...")
            content = repair_truncated_json(content)
        
        # Try to extract JSON with improved logic
        try:
            data = json.loads(content)
            print(f"🔍 Parsed JSON type: {type(data)}")
            
            if isinstance(data, list):
                df = pd.DataFrame(data)
                print(f"📊 Direct array: {len(df)} rows")
            elif isinstance(data, dict):
                # Check for common keys that might contain the data
                for key in ['responses', 'rows', 'data', 'items', 'records', 'results', 'survey_responses']:
                    if key in data and isinstance(data[key], list):
                        df = pd.DataFrame(data[key])
                        print(f"📊 Found data in '{key}': {len(df)} rows")
                        break
                else:
                    # If no standard key found, check if all values are lists/objects
                    list_keys = [k for k, v in data.items() if isinstance(v, list) and len(v) > 0]
                    if list_keys:
                        # Use the first list key found
                        key = list_keys[0]
                        df = pd.DataFrame(data[key])
                        print(f"📊 Found data in '{key}': {len(df)} rows")
                    else:
                        # Try to convert the dict values to a list
                        if all(isinstance(v, dict) for v in data.values()):
                            df = pd.DataFrame(list(data.values()))
                            print(f"📊 Converted dict values: {len(df)} rows")
                        else:
                            raise ValueError(f"Unexpected JSON structure: {list(data.keys())}")
            else:
                raise ValueError(f"Unexpected JSON type: {type(data)}")
            
            if len(df) == n_rows:
                print(f"✅ Successfully generated {len(df)} survey responses")
                return df
            else:
                print(f"⚠️ Generated {len(df)} rows, expected {n_rows}")
                if len(df) > 0:
                    return df
                else:
                    raise ValueError("No data generated")
                    
        except json.JSONDecodeError as e:
            print(f"❌ JSON parsing failed: {str(e)}")
            # Try the improved extract_strict_json function
            try:
                data = extract_strict_json(content)
                df = pd.DataFrame(data)
                print(f"✅ Recovered with strict parsing: {len(df)} rows")
                return df
            except Exception as e2:
                print(f"❌ Strict parsing also failed: {str(e2)}")
                # Print a sample of the content for debugging
                print(f"🔍 Content sample: {content[:500]}...")
                raise e2
                
    except Exception as e:
        print(f'❌ LLM error, fallback to rule-based mock: {str(e)}')
        tmp = dict(CFG); tmp['rows'] = n_rows
        return generate_rule_based(tmp)

def fixed_generate_llm(CFG, total_rows=200, batch_size=50):
    """Fixed LLM generation with adaptive batch processing"""
    print(f"🚀 Generating {total_rows} survey responses with adaptive batching")
    
    # Adaptive batch sizing based on total rows
    if total_rows <= 20:
        optimal_batch_size = min(batch_size, total_rows)
    elif total_rows <= 50:
        optimal_batch_size = min(15, batch_size)
    elif total_rows <= 100:
        optimal_batch_size = min(10, batch_size)
    else:
        optimal_batch_size = min(8, batch_size)
    
    print(f"📊 Using optimal batch size: {optimal_batch_size}")
    
    all_dataframes = []
    remaining = total_rows
    
    while remaining > 0:
        current_batch_size = min(optimal_batch_size, remaining)
        print(f"\n📦 Processing batch: {current_batch_size} rows (remaining: {remaining})")
        
        try:
            batch_df = fixed_llm_generate_batch(CFG, current_batch_size)
            all_dataframes.append(batch_df)
            remaining -= len(batch_df)
            
            # Small delay between batches to avoid rate limits
            if remaining > 0:
                time.sleep(1.5)
                
        except Exception as e:
            print(f"❌ Batch failed: {str(e)}")
            print(f"🔄 Retrying with smaller batch size...")
            
            # Try with smaller batch size
            smaller_batch = max(1, current_batch_size // 2)
            if smaller_batch < current_batch_size:
                try:
                    print(f"🔄 Retrying with {smaller_batch} rows...")
                    batch_df = fixed_llm_generate_batch(CFG, smaller_batch)
                    all_dataframes.append(batch_df)
                    remaining -= len(batch_df)
                    continue
                except Exception as e2:
                    print(f"❌ Retry also failed: {str(e2)}")
            
            print(f"Using rule-based fallback for remaining {remaining} rows")
            fallback_df = generate_rule_based(CFG, remaining)
            all_dataframes.append(fallback_df)
            break
    
    if all_dataframes:
        result = pd.concat(all_dataframes, ignore_index=True)
        print(f"✅ Generated total: {len(result)} survey responses")
        return result
    else:
        print("❌ No data generated")
        return pd.DataFrame()



In [ ]:
# Test the fixed LLM generation
print("🧪 Testing LLM generation...")

# Test with small dataset first
test_df = fixed_llm_generate_batch(CFG, 10)
print(f"\n📊 Generated dataset shape: {test_df.shape}")
print(f"\n📋 First few rows:")
print(test_df.head())
print(f"\n📈 Data types:")
print(test_df.dtypes)

# Debug function to see what the LLM is actually returning
def debug_llm_response(CFG, n_rows=5):
    """Debug function to see raw LLM response"""
    if not os.getenv('OPENAI_API_KEY'):
        print("No OpenAI API key available for debugging")
        return
    
    try:
        from openai import OpenAI
        client = OpenAI()
        
        prompt = create_survey_prompt(CFG, n_rows)
        
        print(f"\n🔍 DEBUG: Testing with {n_rows} rows")
        print(f"📝 Prompt length: {len(prompt)} characters")
        
        response = client.chat.completions.create(
            model='gpt-4o-mini',
            messages=[
                {'role': 'system', 'content': 'You are a data generation expert. Generate realistic survey data in JSON format.'},
                {'role': 'user', 'content': prompt}
            ],
            temperature=0.3,
            max_tokens=2000,
            response_format={'type': 'json_object'}
        )
        
        content = response.choices[0].message.content
        print(f"📝 Raw response length: {len(content)} characters")
        print(f"🔍 First 200 characters: {content[:200]}")
        print(f"🔍 Last 200 characters: {content[-200:]}")
        
        # Try to parse
        try:
            data = json.loads(content)
            print(f"✅ JSON parsed successfully")
            print(f"🔍 Data type: {type(data)}")
            if isinstance(data, dict):
                print(f"🔍 Dict keys: {list(data.keys())}")
            elif isinstance(data, list):
                print(f"🔍 List length: {len(data)}")
        except Exception as e:
            print(f"❌ JSON parsing failed: {str(e)}")
            
    except Exception as e:
        print(f"❌ Debug failed: {str(e)}")


In [ ]:
# Test the fixed implementation
print("🧪 Testing the fixed LLM generation...")

# Test with small dataset
test_df = fixed_llm_generate_batch(CFG, 5)
print(f"\n📊 Generated dataset shape: {test_df.shape}")
print(f"\n📋 First few rows:")
print(test_df.head())
print(f"\n📈 Data types:")
print(test_df.dtypes)

if not test_df.empty:
    print(f"\n✅ SUCCESS! LLM generation is now working!")
    print(f"📊 Generated {len(test_df)} survey responses using LLM")
else:
    print(f"\n❌ Still having issues with LLM generation")


In [ ]:
#Test larger dataset generation 
print("🚀 Testing larger dataset generation...")
large_df = fixed_generate_llm(CFG, total_rows=100, batch_size=25)
if not large_df.empty:
    print(f"\n📊 Large dataset shape: {large_df.shape}")
    print(f"\n📈 Summary statistics:")
    print(large_df.describe())
    
    # Save the results
    from pathlib import Path
    out = Path("data"); out.mkdir(exist_ok=True)
    ts = datetime.utcnow().strftime("%Y%m%dT%H%M%SZ")
    csv_path = out / f"survey_llm_fixed_{ts}.csv"
    large_df.to_csv(csv_path, index=False)
    print(f"💾 Saved: {csv_path}")


In [ ]:

def build_json_schema(CFG):
    schema = {'type':'array','items':{'type':'object','properties':{},'required':[]}}
    props = schema['items']['properties']; req = schema['items']['required']
    for f in CFG['fields']:
        name, t = f['name'], f['type']
        req.append(name)
        if t in ('int','float'): props[name] = {'type':'number' if t=='float' else 'integer'}
        elif t == 'enum': props[name] = {'type':'string','enum': f['values']}
        elif t in ('uuid4','datetime'): props[name] = {'type':'string'}
        elif t == 'bool': props[name] = {'type':'boolean'}
        else: props[name] = {'type':'string'}
    return schema

PROMPT_PREAMBLE = (
    "You are a data generator. Return ONLY JSON. "
    "Respond as a JSON object with key 'rows' whose value is an array of exactly N objects. "
    "No prose, no code fences, no trailing commas."
)

def render_prompt(CFG, n_rows=100):
    minimal_cfg = {'fields': []}
    for f in CFG['fields']:
        base = {k: f[k] for k in ['name','type'] if k in f}
        if 'min' in f and 'max' in f: base.update({'min': f['min'], 'max': f['max']})
        if 'values' in f: base.update({'values': f['values']})
        if 'fmt' in f: base.update({'fmt': f['fmt']})
        minimal_cfg['fields'].append(base)
    return {
        'preamble': PROMPT_PREAMBLE,
        'n_rows': n_rows,
        'schema': build_json_schema(CFG),
        'constraints': minimal_cfg,
        'instruction': f"Return ONLY this structure: {{'rows': [ ... exactly {n_rows} objects ... ]}}"
    }

def parse_llm_json_to_df(raw: str) -> pd.DataFrame:
    try:
        obj = json.loads(raw)
        if isinstance(obj, dict) and isinstance(obj.get('rows'), list):
            return pd.DataFrame(obj['rows'])
    except Exception:
        pass
    data = extract_strict_json(raw)
    return pd.DataFrame(data)

USE_LLM = bool(os.getenv('OPENAI_API_KEY'))
print('LLM available:', USE_LLM)

def llm_generate_batch(CFG, n_rows=50):
    if USE_LLM:
        try:
            from openai import OpenAI
            client = OpenAI()
            prompt = json.dumps(render_prompt(CFG, n_rows))
            resp = client.chat.completions.create(
                model='gpt-4o-mini',
                response_format={'type': 'json_object'},
                messages=[
                    {'role':'system','content':'You output strict JSON only.'},
                    {'role':'user','content': prompt}
                ],
                temperature=0.2,
                max_tokens=8192,
            )
            raw = resp.choices[0].message.content
            try:
                return parse_llm_json_to_df(raw)
            except Exception:
                stricter = (
                    prompt
                    + "\nReturn ONLY a JSON object structured as: "
                    + "{\"rows\": [ ... exactly N objects ... ]}. "
                    + "No prose, no explanations."
                )
                resp2 = client.chat.completions.create(
                    model='gpt-4o-mini',
                    response_format={'type': 'json_object'},
                    messages=[
                        {'role':'system','content':'You output strict JSON only.'},
                        {'role':'user','content': stricter}
                    ],
                    temperature=0.2,
                    max_tokens=8192,
                )
                raw2 = resp2.choices[0].message.content
                return parse_llm_json_to_df(raw2)
        except Exception as e:
            print('LLM error, fallback to rule-based mock:', e)
    tmp = dict(CFG); tmp['rows'] = n_rows
    return generate_rule_based(tmp)

def generate_llm(CFG, total_rows=200, batch_size=50):
    dfs = []; remaining = total_rows
    while remaining > 0:
        b = min(batch_size, remaining)
        dfs.append(llm_generate_batch(CFG, n_rows=b))
        remaining -= b
        time.sleep(0.2)
    return pd.concat(dfs, ignore_index=True)



In [ ]:
df_llm = generate_llm(CFG, total_rows=100, batch_size=50)
df_llm.head()

In [ ]:
# Test the improved LLM generation with adaptive batching
print("🧪 Testing improved LLM generation with adaptive batching...")

# Test with smaller dataset first
print("\n📦 Testing small batch (10 rows)...")
small_df = fixed_llm_generate_batch(CFG, 10)
print(f"✅ Small batch result: {len(small_df)} rows")

# Test with medium dataset using adaptive batching
print("\n📦 Testing medium dataset (30 rows) with adaptive batching...")
medium_df = fixed_generate_llm(CFG, total_rows=30, batch_size=15)
print(f"✅ Medium dataset result: {len(medium_df)} rows")

if not medium_df.empty:
    print(f"\n📊 Dataset shape: {medium_df.shape}")
    print(f"\n📋 First few rows:")
    print(medium_df.head())
    
    # Save the results
    from pathlib import Path
    out = Path("data"); out.mkdir(exist_ok=True)
    ts = datetime.utcnow().strftime("%Y%m%dT%H%M%SZ")
    csv_path = out / f"survey_adaptive_batch_{ts}.csv"
    medium_df.to_csv(csv_path, index=False)
    print(f"💾 Saved: {csv_path}")
else:
    print("❌ Medium dataset generation failed")
